# LEGO Element Step Finder
This notebook allows you to find all steps where a specific LEGO element appears across all sets.

## Setup and Usage
1. Make sure your DATABASE_URL environment variable is set
2. Enter an element ID in the input field below
3. Run the cell to see all steps where this element appears

## Imports

In [1]:
import os
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, inspect
from IPython.display import display
from pathlib import Path
import sys

# Interactive element search
from ipywidgets import widgets
from IPython.display import display, clear_output

# Get the absolute path of the current directory
current_dir = Path().resolve()
# If we're in the notebooks or src directory, move up one level
# to the project root directory
project_root = current_dir.parent if current_dir.name in ['notebooks', 'src'] else current_dir
# Add the project root directory to Python's path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
# Add /src to Python's path
src_dir = project_root / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

# Change the current working directory
os.chdir(project_root)

In [6]:
def get_steps_by_element_id(element_id, db_url=None):
    """
    Get all steps where a specific LEGO element appears.
    
    Args:
        element_id (str): The element ID to search for
        db_url (str, optional): Database URL. If None, uses DATABASE_URL env variable
        
    Returns:
        dict: Dictionary with set numbers as keys and lists of step information as values
    """
    if db_url is None:
        db_url = os.environ.get('DATABASE_URL')
        if not db_url:
            raise ValueError("No database URL provided and DATABASE_URL env variable not set")
    
    # Create engine based on URL type
    if db_url.startswith('sqlite:'):
        engine = create_engine(db_url)
    else:
        # For demonstration, assuming SQLite
        conn = sqlite3.connect(db_url)
    
    # SQL query to get all steps for a specific element
    query = """
    SELECT 
        s.set_num,
        ss.booklet_number,
        ss.page_number,
        ss.step_number
    FROM 
        step_elements se
    JOIN 
        set_steps ss ON se.step_id = ss.step_id
    JOIN 
        inventories i ON ss.inventory_id = i.id
    JOIN 
        sets s ON i.set_num = s.set_num
    WHERE 
        se.element_id = :element_id
    ORDER BY 
        s.set_num, 
        ss.booklet_number, 
        ss.page_number, 
        ss.step_number
    """
    
    try:
        # Execute query
        if 'engine' in locals():
            df = pd.read_sql_query(query, engine, params={"element_id": element_id})
        else:
            df = pd.read_sql_query(query, conn, params={"element_id": element_id})
        
        return df
    
    except Exception as e:
        print(f"Error executing query: {e}")
        return {}
    finally:
        if 'conn' in locals():
            conn.close()
        if 'engine' in locals():
            engine.dispose()

In [12]:
def search_element(element_id):
    
    db_url = "sqlite:///test_db.db"

    print(f"Searching for element ID: {element_id}")
    df = get_steps_by_element_id(element_id, db_url)
    
    if df.empty:
        print("No steps found for this element ID")
        return
    
    sets = df['set_num'].unique()
    print(f"Found steps in {len(df['set_num'].unique())} set(s):")
    for s in sets:
        display(df[df['set_num'] == s])

search_element('302421')

Searching for element ID: 302421
Found steps in 1 set(s):


,set_num,booklet_number,page_number,step_number
0,31147-1,1,16,18
1,31147-1,1,25,26
2,31147-1,1,30,31
3,31147-1,2,9,12
4,31147-1,2,15,19
5,31147-1,2,16,20
6,31147-1,3,4,4
7,31147-1,3,17,14


In [15]:
def debug_search(element_id):
    print(f"Testing with element ID: {element_id}")

    try:
        # For debugging
        db_url = "sqlite:///test_db.db"
        #db_url = os.environ.get('DATABASE_URL')
        print(f"Using DB URL: {db_url}")

        # Create engine based on URL type
        if db_url.startswith('sqlite:'):
            engine = create_engine(db_url)
            inspector = inspect(engine)
            # Get all table names
            tables = inspector.get_table_names()

            # For SQLAlchemy, parameters need to be passed differently
            print("Found the following tables: ", tables)

            df = pd.read_sql_query(
                "SELECT * FROM step_elements WHERE element_id = :element_id LIMIT 5",
                engine,
                params={"element_id": element_id}
            )
        else:
            conn = sqlite3.connect(db_url)
            # For sqlite3 connection, parameters can be passed as a list
            df = pd.read_sql_query(
                "SELECT * FROM step_elements WHERE element_id = ? LIMIT 5",
                conn,
                params=(element_id,)  # Note the comma to make it a tuple
            )

        print("Query executed successfully")
        print(f"Found {len(df)} rows")
        print(df.head())

        # Continue with the full query if the simple one works
        return True
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return False

debug_search("302421")

Testing with element ID: 302421
Using DB URL: sqlite:///test_db.db
Found the following tables:  ['set_steps', 'step_elements']
Query executed successfully
Found 2 rows
   step_id element_id
0        7     302421
1       12     302421


True